-> Initial high level results

Time period:		
--------# users	GMV / User    
Control 		
Treatment		

In [3]:
from pyhive import presto
import operator

def get_presto_conn():
    return presto.connect(host='presto-alpha-backend.data.houzz.net', port=8086)

def presto_query(query, presto_conn, fetchall=True):
    cursor = presto_conn.cursor()
    cursor.execute(query)
    header = list(map(operator.itemgetter(0), cursor.description))
    return header, cursor.fetchall()

conn = get_presto_conn() # establish the connection

# Or if want to use df format

import pandas as pd

#df = pd.read_sql_query(q, conn)

### 10 days attribution

GMV/ Checkout/ Orders

* Getting last email date and attributed post 7 days

In [57]:
q = '''
with raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v1%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
  )
--v2: treatment
union all
(select 'treatment1' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
--v3: treatment1
union all
(select 'treatment2' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v3%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
), preprocess as(
  select r1.*, r2.first_email_dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.campaign_uuid != r2.campaign_uuid
        and date_diff('day',date(r1.last_email_dt),date(r2.first_email_dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(last_email_dt) + interval '7' day as varchar)
      else cast(last_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,count(*) campaign_cnt
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.checkout_id) checkout_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.first_email_dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-11-08'
where b.user_id not in (
              select p1.user_id
              from raw p1
                inner join raw p2
                  on p1.user_id = p2.user_id
                    and p2.user_group = 'treatment'
              where p1.user_group = 'control'
            )
group by 1
'''

In [59]:
%%time
result = pd.read_sql_query(q, conn)

result

CPU times: user 70.1 ms, sys: 87.1 ms, total: 157 ms
Wall time: 39.6 s


,user_group,campaign_cnt,user_cnt,checkout_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,control,14340,12018,1333,1174,1830,765622.20,66
1,treatment1,10585,9100,1127,972,1525,683216.27,185
2,treatment2,13285,11131,1369,1173,1869,880902.07,157


In [60]:
result.to_clipboard()

In [61]:
q = '''
with raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v1%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
  )
--v2: treatment
union all
(select 'treatment1' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
--v3: treatment1
union all
(select 'treatment2' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v3%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
), preprocess as(
  select r1.*, r2.first_email_dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.campaign_uuid != r2.campaign_uuid
        and date_diff('day',date(r1.last_email_dt),date(r2.first_email_dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(last_email_dt) + interval '7' day as varchar)
      else cast(last_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,first_email_dt
    ,count(*) campaign_cnt
    ,count(distinct b.user_id) user_cnt
    ,count(distinct m.user_id) xo_user_cnt
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.first_email_dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-11-08'
where b.user_id not in (
              select p1.user_id
              from raw p1
                inner join raw p2
                  on p1.user_id = p2.user_id
                    and p2.user_group = 'treatment'
              where p1.user_group = 'control'
            )
group by 1,2
order by 2,1
'''

In [62]:
%%time
trend = pd.read_sql_query(q, conn)

trend

CPU times: user 60.6 ms, sys: 24.7 ms, total: 85.3 ms
Wall time: 36.3 s


,user_group,first_email_dt,campaign_cnt,user_cnt,xo_user_cnt,order_cnt,gmv,unsub
0,control,2021-11-08,1003,827,38,63,58540.74,1
1,treatment1,2021-11-08,1095,953,62,112,47025.16,6
2,treatment2,2021-11-08,95,43,12,64,31682.53,0
3,control,2021-11-09,2123,1922,164,269,104067.52,3
4,treatment1,2021-11-09,2581,2341,202,364,154766.41,91
5,treatment2,2021-11-09,1400,1152,171,339,154804.47,20
6,control,2021-11-10,911,772,91,178,53320.82,4
7,treatment1,2021-11-10,673,606,73,106,46208.14,7
8,treatment2,2021-11-10,896,763,88,163,76448.46,9
9,control,2021-11-11,279,240,34,61,22787.70,6


In [64]:
trend[trend['first_email_dt']>='2021-11-09'].to_clipboard()

In [65]:
q = '''
with raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v1%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
  )
--v2: treatment
union all
(select 'treatment1' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
--v3: treatment1
union all
(select 'treatment2' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v3%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
), preprocess as(
  select r1.*, r2.first_email_dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.campaign_uuid != r2.campaign_uuid
        and date_diff('day',date(r1.last_email_dt),date(r2.first_email_dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(last_email_dt) + interval '7' day as varchar)
      else cast(last_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,b.user_id
    ,campaign_uuid
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
    ,sum(b.unsub) unsub
from processed b
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and b.first_email_dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and created >= '2021-11-08'
where b.user_id not in (
              select p1.user_id
              from raw p1
                inner join raw p2
                  on p1.user_id = p2.user_id
                    and p2.user_group = 'treatment'
              where p1.user_group = 'control'
            )
group by 1,2,3
'''

In [66]:
%%time
sample = pd.read_sql_query(q, conn)

sample

CPU times: user 264 ms, sys: 654 ms, total: 918 ms
Wall time: 48.9 s


,user_group,user_id,campaign_uuid,order_cnt,gmv,unsub
0,treatment2,70850356,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,0,NaN,0
1,treatment2,60383286,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,0,NaN,0
2,treatment2,57405414,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,0,NaN,0
3,treatment2,1073116,a08af1d4-35a4-40e0-a8d3-9b4149cc1181,0,NaN,0
4,treatment2,70395812,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,0,NaN,0
...,...,...,...,...,...,...
35931,treatment1,9318282,bf8de4f7-ba8c-45b2-a69e-11f57a2dfc9e,0,NaN,0
35932,treatment1,43067230,bf8de4f7-ba8c-45b2-a69e-11f57a2dfc9e,2,2680.47,0
35933,treatment1,8937772,477b671d-17e7-4699-bf8d-0577642ff221,0,NaN,0
35934,treatment1,7276822,477b671d-17e7-4699-bf8d-0577642ff221,0,NaN,0


In [67]:
sample['gmv'].fillna(0,inplace=True)

In [48]:
from scipy.stats import ttest_ind

In [68]:
ttest_ind(sample[sample['user_group']=='control']['gmv'],sample[sample['user_group']=='treatment1']['gmv']).pvalue

0.014059143071286037

In [69]:
ttest_ind(sample[sample['user_group']=='control']['order_cnt'],sample[sample['user_group']=='treatment1']['order_cnt']).pvalue

0.008589292835467744

In [70]:
ttest_ind(sample[sample['user_group']=='control']['gmv'],sample[sample['user_group']=='treatment2']['gmv']).pvalue

0.006897252587482647

In [71]:
ttest_ind(sample[sample['user_group']=='control']['order_cnt'],sample[sample['user_group']=='treatment2']['order_cnt']).pvalue

0.020565073651840664

In [72]:
ttest_ind(sample[sample['user_group']=='control']['unsub'],sample[sample['user_group']=='treatment1']['unsub']).pvalue

0.00010416341977930703

In [73]:
ttest_ind(sample[sample['user_group']=='control']['unsub'],sample[sample['user_group']=='treatment2']['unsub']).pvalue

0.0007459504254783222

In [74]:
sample = sample.groupby(['user_group','user_id'])[['order_cnt','gmv']].sum().reset_index()

In [75]:
ttest_ind(sample[sample['user_group']=='control']['gmv'],sample[sample['user_group']=='treatment1']['gmv']).pvalue

0.06924360320555378

In [76]:
ttest_ind(sample[sample['user_group']=='control']['order_cnt'],sample[sample['user_group']=='treatment1']['order_cnt']).pvalue

0.05611537226313603

In [77]:
ttest_ind(sample[sample['user_group']=='control']['gmv'],sample[sample['user_group']=='treatment2']['gmv']).pvalue

0.023235374278968452

In [78]:
ttest_ind(sample[sample['user_group']=='control']['order_cnt'],sample[sample['user_group']=='treatment2']['order_cnt']).pvalue

0.04817217109336739

In [81]:
q = '''
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,b2.trigger_name
    ,count(b1.event_id) delivered
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v1%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
  )
--v2: treatment
union all
(select 'treatment1' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,b2.trigger_name
    ,count(b1.event_id) delivered
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
--v3: treatment1
union all
(select 'treatment2' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,b2.trigger_name
    ,count(b1.event_id) delivered
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v3%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3,4
)
'''

In [82]:
%%time
check = pd.read_sql_query(q, conn)

check

CPU times: user 614 ms, sys: 49.4 ms, total: 663 ms
Wall time: 30.5 s


,user_group,user_id,campaign_uuid,trigger_name,delivered,unsub
0,control,733964,2952ce0e-bfa2-4337-892e-038706293b3f,Day 2,1,0
1,control,63463377,2952ce0e-bfa2-4337-892e-038706293b3f,Day 7,3,0
2,control,14426712,961c12e9-0917-496d-be7e-e2b2a28f7f40,Day 0,13,0
3,control,14640167,2952ce0e-bfa2-4337-892e-038706293b3f,Day 0,2,0
4,control,40208621,961c12e9-0917-496d-be7e-e2b2a28f7f40,Day 0,6,0
...,...,...,...,...,...,...
124794,treatment2,41692359,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,D0,1,0
124795,treatment2,64260561,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,D3,2,0
124796,treatment2,52044364,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,D3,2,0
124797,treatment2,56924926,541f0ae0-7c8b-4bf6-aacf-a8b254ae96e0,D0,2,0


In [86]:
check.groupby(['user_group','trigger_name'])[['delivered','unsub']].sum().to_clipboard()

In [85]:
#check.to_clipboard()

### Check user behaviors among treatment2:
    - Account Age
    - How they react

In [136]:
q = '''
select b.user_id
    ,trigger_name
    ,case when lower(b.bsft_link_url) like '%optout%' then 'unsubscribe'
        else event_name end as event_name
    ,substring(c.created, 1, 7) as create_yyyymm
    ,b.dt
    ,m.created
    ,count(distinct m.order_id) order_cnt
    ,sum(m.gmv) gmv
from bls.blueshift_events b
    inner join c2.users c
        on b.user_id = cast(c.user_id as varchar)
    left join mp.order_metrics_with_adjustments m
        on b.user_id = cast(m.user_id as varchar)
            and order_status in (0,1,2,3,4,5,6,20,99)
            and is_replacement_order = 0
            and acc_class = '10 - Consumer'
            and m.created >= c.created
where b.dt between '2021-11-08' and '2021-11-29'
  and lower(b.campaign_name) like '%cart%' 
  and lower(b.campaign_name) like '%v3%'
group by 1,2,3,4,5,6
'''

In [137]:
%%time
result = pd.read_sql_query(q, conn)

result

CPU times: user 1.53 s, sys: 154 ms, total: 1.68 s
Wall time: 39.3 s


,user_id,trigger_name,event_name,create_yyyymm,dt,created,order_cnt,gmv
0,40619530,D3,Email Delivered,2017-04,2021-11-12,2017-05-17 19:03:35.0,1,354.00
1,60062550,D1,Email Delivered,2020-02,2021-11-12,None,0,NaN
2,3216154,D1,Email Delivered,2012-09,2021-11-12,2021-04-02 06:23:59.0,1,430.55
3,3216154,D1,Push Notification Sent,2012-09,2021-11-12,2021-11-13 08:48:04.0,2,1566.04
4,3216154,D1,Push Notification Sent to Device,2012-09,2021-11-12,2021-02-24 06:55:54.0,1,271.24
...,...,...,...,...,...,...,...,...
270939,7716193,D3,Email Delivered,2013-06,2021-11-19,2018-01-24 10:46:27.0,1,125.96
270940,7716193,D3,Email Delivered,2013-06,2021-11-19,2018-05-09 13:54:30.0,1,467.44
270941,7716193,D0,Email Opened,2013-06,2021-11-19,2021-11-02 15:10:23.0,1,399.00
270942,9377992,D5,Email Delivered,2013-08,2021-11-19,2021-02-05 14:11:41.0,2,89.00


In [119]:
result['event_name'].value_counts()

Email Delivered                         45030
Email Opened                            23157
Push Notification Sent to Device        13534
Push Notification Sent                  11310
Push Notification Delivered             10406
Email Link Clicked                       3180
Push Notification Link Clicked            765
Push Notification Bounced for Device      287
Push Notification Bounced                 249
unsubscribe                                64
Email Bounced                              36
Name: event_name, dtype: int64

In [116]:
pd.pivot_table(result, values='create_yyyymm', index=['event_name'],columns=['trigger_name']
    , aggfunc='count').T[['Email Delivered','Email Opened','Email Link Clicked','unsubscribe']].to_clipboard()

In [115]:
pd.pivot_table(result, values='create_yyyymm', index=['event_name'],columns=['trigger_name']
    , aggfunc='count').T[['Push Notification Delivered','Push Notification Link Clicked']]

event_name,Push Notification Delivered,Push Notification Link Clicked
trigger_name,,
D0,2983,346
D1,2711,176
D3,2477,139
D5,2235,104


In [147]:
sum(result['user_id'].value_counts()>1)

13369

In [148]:
sum(result[result['event_name']=='unsubscribe']['user_id'].value_counts()>1)

13

### Abandoned Products vs Others (Same categories or else)

In [173]:
q = '''
with raw as (
--v1: control
  (select 'control' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v1%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
  )
--v2: treatment
union all
(select 'treatment1' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v2%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
--v3: treatment1
union all
(select 'treatment2' as user_group
    ,b1.user_id
    ,b1.campaign_uuid
    ,min(b1.dt) first_email_dt
    ,max(b1.dt) last_email_dt
    ,sum(case when lower(b2.bsft_link_url) like '%optout%' then 1 else 0 end) as unsub
from bls.blueshift_events b1
    left join bls.blueshift_events b2
        on b1.user_id = b2.user_id
            and b1.campaign_uuid = b2.campaign_uuid
            and b1.campaign_name = b2.campaign_name
where b1.dt between '2021-11-08' and '2021-11-22'
  and lower(b1.campaign_name) like '%cart%' 
  and lower(b1.campaign_name) like '%v3%'
  and (b1.event_name = 'Email Delivered' or b1.event_name = 'Push Notification Delivered')
group by 1,2,3
)
), preprocess as(
  select r1.*, r2.first_email_dt as next_email_dt
  from raw r1
    left join raw r2
      on r1.user_id = r2.user_id 
        and r1.campaign_uuid != r2.campaign_uuid
        and date_diff('day',date(r1.last_email_dt),date(r2.first_email_dt)) between 0 and 7
), processed as(
  select *
    , case when next_email_dt is null then cast(date(last_email_dt) + interval '7' day as varchar)
      else cast(last_email_dt as varchar) end as attributed_last_day
  from preprocess
)
select user_group
    ,b.user_id
    ,b.first_email_dt
    ,m.created
    ,m.order_id
    ,json_extract(l.action_metadata , '$.vlid') atc_product
    ,json_extract(l.action_metadata , '$.uprice') atc_price
    ,m.item_id purchaed_products
    ,m.product_price
    ,m.shipping
    ,case when cast(json_extract(l.action_metadata , '$.vlid') as int) = m.item_id then 'abandoned_products'
        else 'other_products' end as purchase_type
    , s_xo.l1_category_name xo_l1
    , s_xo.l2_category_name xo_l2
    , s_atc.l1_category_name atc_l1
    , s_atc.l2_category_name atc_l2
from processed b
    inner join mp.order_item_margins_with_replacement m
        on b.user_id = cast(m.user_id as varchar)
            and b.first_email_dt <= substring(m.created,1,10)
            and b.attributed_last_day >= substring(m.created,1,10)
            and status in (0, 1, 2, 3, 4, 5, 20, 99)
            and is_replacement_order = 0
            and created >= '2021-11-08'
    inner join l2.user_action l
        on m.visitor_id = l.visitor_id
            and b.first_email_dt = l.dt
            and l.action = 'CART_ADD'
    inner join shop.vl_pupil s_xo
        on m.item_id = s_xo.vendor_listing_id
    inner join shop.vl_pupil s_atc
        on cast(json_extract(l.action_metadata , '$.vlid') as int) = s_atc.vendor_listing_id
where l.dt >= '2021-11-08'
'''

In [174]:
%%time
result = pd.read_sql_query(q, conn)

result

CPU times: user 199 ms, sys: 33.2 ms, total: 232 ms
Wall time: 1min 2s


,user_group,user_id,first_email_dt,created,order_id,atc_product,atc_price,purchaed_products,product_price,shipping,purchase_type,xo_l1,xo_l2,atc_l1,atc_l2
0,treatment2,67138435,2021-11-16,2021-11-20 14:42:48.0,1716988850284811,113739342,"""121.99""",113739342,121.99,0.000,abandoned_products,furniture,bedroom furniture,furniture,bedroom furniture
1,treatment2,67138435,2021-11-16,2021-11-20 14:42:48.0,1716988850284811,113739342,"""121.99""",113739342,121.99,0.000,abandoned_products,furniture,bedroom furniture,furniture,bedroom furniture
2,control,67141548,2021-11-19,2021-11-19 17:38:52.0,1716909331346147,111782101,"""1300.00""",111782101,1300.00,0.000,abandoned_products,bath products,bathroom vanities and sink consoles,bath products,bathroom vanities and sink consoles
3,treatment2,70003854,2021-11-19,2021-11-19 11:17:46.0,1716885353928825,94034582,"""60.00""",94034582,60.00,0.000,abandoned_products,home improvement,heating and cooling,home improvement,heating and cooling
4,treatment1,27966081,2021-11-12,2021-11-17 07:46:06.0,1716690843733579,93712996,"""28.99""",93712996,28.99,1.975,abandoned_products,home decor,holiday decorations,home decor,holiday decorations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13340,control,12284261,2021-11-19,2021-11-23 06:33:41.0,1717229869074389,116142006,"""460.42""",41829857,352.00,0.000,other_products,outdoor products,patio furniture and outdoor furniture,outdoor products,patio furniture and outdoor furniture
13341,treatment2,29772942,2021-11-16,2021-11-18 14:55:35.0,1716808461103606,107213704,"""189.99""",107213704,189.99,0.000,abandoned_products,bath products,bathroom accessories,bath products,bathroom accessories
13342,treatment2,29772942,2021-11-16,2021-11-18 15:00:16.0,1716808755260147,107213704,"""189.99""",3529839,75.99,0.000,other_products,bath products,bathroom accessories,bath products,bathroom accessories
13343,treatment1,53179339,2021-11-16,2021-11-17 10:04:01.0,1716699519815814,117213871,"""510.00""",117213871,510.00,0.000,abandoned_products,furniture,living room furniture,furniture,living room furniture


In [175]:
result['atc_price'] = result['atc_price'].apply(lambda x: float(x[1:-1]))

In [176]:
result['price_diff'] = result.apply(lambda x: x['product_price'] - x['atc_price'],axis=1)

In [177]:
result_abandon = result[result['purchase_type']!='other_products']

In [178]:
result_abandon.drop_duplicates(inplace=True)

<ipython-input-178-e69b09ec31c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_abandon.drop_duplicates(inplace=True)


In [179]:
result_abandon.groupby('user_group').count()

,user_id,first_email_dt,created,order_id,atc_product,atc_price,purchaed_products,product_price,shipping,purchase_type,xo_l1,xo_l2,atc_l1,atc_l2,price_diff
user_group,,,,,,,,,,,,,,,
control,454,454,454,454,454,454,454,454,454,454,448,448,448,448,454
treatment1,353,353,353,353,353,353,353,353,353,353,344,344,344,344,353
treatment2,460,460,460,460,460,460,460,460,460,460,457,457,457,457,460


In [180]:
result_dedupe = result_abandon.append(result[result['purchase_type']=='other_products'])

In [181]:
result_dedupe[:3]

,user_group,user_id,first_email_dt,created,order_id,atc_product,atc_price,purchaed_products,product_price,shipping,purchase_type,xo_l1,xo_l2,atc_l1,atc_l2,price_diff
0,treatment2,67138435,2021-11-16,2021-11-20 14:42:48.0,1716988850284811,113739342,121.99,113739342,121.99,0.0,abandoned_products,furniture,bedroom furniture,furniture,bedroom furniture,0.0
2,control,67141548,2021-11-19,2021-11-19 17:38:52.0,1716909331346147,111782101,1300.00,111782101,1300.00,0.0,abandoned_products,bath products,bathroom vanities and sink consoles,bath products,bathroom vanities and sink consoles,0.0
3,treatment2,70003854,2021-11-19,2021-11-19 11:17:46.0,1716885353928825,94034582,60.00,94034582,60.00,0.0,abandoned_products,home improvement,heating and cooling,home improvement,heating and cooling,0.0


In [187]:
result_dedupe['similar_products'] = 0
result_dedupe.loc[(result_dedupe['purchase_type']!='abandoned_products')
              &(result_dedupe['xo_l2'] == result_dedupe['atc_l2']),'similar_products'] = 1

result_dedupe['related_products'] = 0
result_dedupe.loc[(result_dedupe['purchase_type']!='abandoned_products')
              &(result_dedupe['xo_l1'] == result_dedupe['atc_l1']),'related_products'] = 1

In [190]:
result_dedupe.shape

(12284, 18)

In [193]:
result_dedupe.drop_duplicates(inplace=True)

In [195]:
result_dedupe.groupby(['user_group','purchase_type']).agg({'atc_price':'mean','product_price':'mean'
    ,'shipping':'mean','price_diff':'mean','user_id':pd.Series.nunique,'order_id':pd.Series.nunique
    ,'first_email_dt':'count','similar_products':'sum','related_products':'sum'})

atc_price  product_price  shipping  \
user_group purchase_type                                             
control    abandoned_products  349.069383     347.554009  2.580764   
           other_products      273.355937     221.784542  1.566767   
treatment1 abandoned_products  316.350142     315.758272  1.549972   
           other_products      367.982314     354.582388  1.153586   
treatment2 abandoned_products  351.011152     350.121370  2.319704   
           other_products      290.457638     292.701500  2.340060   

                               price_diff  user_id  order_id  first_email_dt  \
user_group purchase_type                                                       
control    abandoned_products   -1.515374      337       427             454   
           other_products      -51.571395      318       605            2860   
treatment1 abandoned_products   -0.591870      258       314             353   
           other_products      -13.399926      279       540            1763   
treatment2 abandoned_products   -0.889783      359       434             460   
           other_products        2.243862      373       658            2087   

                               similar_products  related_products  
user_group purchase_type                                           
control    abandoned_products                 0                 0  
           other_products                  1110              1644  
treatment1 abandoned_products                 0                 0  
           other_products                   703               956  
treatment2 abandoned_products                 0                 0  
           other_products                   835              1171

In [196]:
result_dedupe.groupby(['user_group','purchase_type']).agg({'atc_price':'mean','product_price':'mean'
    ,'shipping':'mean','price_diff':'mean','user_id':pd.Series.nunique,'order_id':pd.Series.nunique
    ,'first_email_dt':'count','similar_products':'sum','related_products':'sum'}).to_clipboard()

### Check Omnilog for recommended products

In [199]:
q = '''
select *
    ,event.event_details
from l2.omnilog_events
where lower(event.event_details) like '%cart%'
    and 
    and dt='2021-11-30'
'''

In [200]:
%%time
omnilog = pd.read_sql_query(q, conn)

omnilog

CPU times: user 2.87 s, sys: 298 ms, total: 3.17 s
Wall time: 1min 26s


,protocol,session,client_info,component,device,client_request,geo_info,experience,flow,client_state,event_source,event,dt,hr,event_details
0,"[1, 1638322874000, ]","[-1651177072661079329, None, None, 34274331, N...","[None, None, None, 502, None, 0]","[Blueshift Webhook, 500, 502, None, 1, None, N...","[5, blueshift, None, None, None, None, 6, None...","[127.0.0.1|399|21_11_30__17_42_20_192|722016, ...","[45.8234, -119.7257, None, None, None]","[None, None, None, None, None, None, None, Non...","[None, None, None, None]","[, None, False, False, None, None, None]","[None, None, None, None, None, None, None, Non...","[1638322874000, 2021-11-30, 17, 9a1e2db4-1b64-...",2021-11-30,17,"{""communication_provider"":""Blueshift"",""ho_user..."
1,"[1, 1638321974287, ]","[628418718659141141, c5288667-b8eb-4571-8654-7...","[aa520d70-d66f-480e-8c59-e16a110309a0, , , 102...","[Jukwaa Web, 100, 102, None, None, None, None,...","[3, None, None, None, None, None, None, None, ...","[12383cea-c9cb-4ce3-986a-5ecc0022c478, 68.107....","[32.7404, -117.1355, None, None, None]","[ckwmulpu700003r677pzjuq3b, Houzz, Houzz US, P...","[None, None, None, None]","[, 1, False, False, 2, 390, 664]","[None, None, None, None, None, None, None, Non...","[1638321974287, 2021-11-30, 17, 9b53a5d9-5f15-...",2021-11-30,17,"{""cart_total_value"":4915.12,""product_ids"":[142..."
2,"[1, 1638320730664, ]","[-3504877358082343451, bbf9c1d8-776e-4d11-b067...","[c0259d6e-1e3c-41df-8a3f-7c35960a707f, None, N...","[Mobile App, 200, 201, 21.11.11, 21, 11, 11, 5...","[3, iPhone, Apple, None, None, None, 4, 14, 8, 1]","[fd38404d-ae5b-4378-9147-a1a0e8ac2471, 184.153...","[40.7653, -73.9589, None, None, None]","[9723a3c0-ae5f-4d86-beae-3d626e169b4e, HOUZZ, ...","[None, None, None, None]","[, 1, False, False, 0, 414, 896]","[None, None, None, None, None, None, None, Non...","[1638320724269, 2021-11-30, 17, 9c7ac114-36da-...",2021-11-30,17,"{""results_count"":""8"",""search_query"":""Bar carts..."
3,"[1, 1638322630000, ]","[7715271558968923991, None, None, 8035313, Non...","[aa812f55-476f-48ec-87d0-b78deaa2418b, None, N...","[Blueshift Webhook, 500, 502, None, 1, None, N...","[5, blueshift, None, None, None, None, 6, None...","[127.0.0.1|7334|21_11_30__17_38_11_700|526811,...","[45.8234, -119.7257, None, None, None]","[None, None, None, None, None, None, None, Non...","[None, None, None, None]","[, None, False, False, None, None, None]","[None, None, None, None, None, None, None, Non...","[1638322630000, 2021-11-30, 17, 9d601330-b760-...",2021-11-30,17,"{""communication_provider"":""Blueshift"",""ho_user..."
4,"[1, 1638322626000, ]","[-7350448637913718747, None, None, 799451, Non...","[None, None, None, 502, None, 0]","[Blueshift Webhook, 500, 502, None, 1, None, N...","[5, blueshift, None, None, None, None, 6, None...",[127.0.0.1|29415|21_11_30__17_38_11_679|242445...,"[45.8234, -119.7257, None, None, None]","[None, None, None, None, None, None, None, Non...","[None, None, None, None]","[, None, False, False, None, None, None]","[None, None, None, None, None, None, None, Non...","[1638322626000, 2021-11-30, 17, 9d8dbd6c-dcbf-...",2021-11-30,17,"{""communication_provider"":""Blueshift"",""ho_user..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41987,"[1, 1638311695038, ]","[-2987186304469163, cd4e7d7d-cddd-403e-8473-64...","[f9a9d819-49d9-41d0-9db9-1732df94c7ca, , , 101...","[C2 Web, 100, 101, None, None, None, None, None]","[1, None, None, None, None, None, None, None, ...","[0b43f763-bd86-4733-9b4b-cba51070bd58, 107.161...","[45.5312, -73.5863, None, None, None]","[ckwmohabr00003e8pxt5w2vbz, Houzz, Houzz US, P...","[ckwmohebi00013e8py7ije5s5, Checkout, None, None]","[, 2, False, False, None, 0, 0]","[A, None, None, None, None, None, None, None, ...","[1638311695038, 2021-11-30, 14, 6fc130c4-a4a6-...",2021-11-30,14,"{""is_from_view_cart"":true,""is_afterpay_enabled..."
41988,"[1, 1638310614000, ]","[8712694422649263703, None, None, 35836337, No...","[82462f5d-030

In [202]:
omnilog['event'][0]

[1638322874000,
 '2021-11-30',
 '17',
 '9a1e2db4-1b64-4048-9c16-81a12fbf0425',
 'Push Notification Sent',
 None,
 0,
 '{"communication_provider":"Blueshift","ho_user_id":"34274331","third_party_user_id":"168d8244-62cc-4035-bba5-6061f120da67","campaign_type":"transactional","campaign_name":"Cart Reminder - V2 - Push","campaign_uuid":"477b671d-17e7-4699-bf8d-0577642ff221","communication_template":"Cart Reminder_V2_D3","trigger_name":"D3","trigger_type":"Push Trigger","trigger_uuid":"952031c5-636e-4c4e-8aa1-50c78e33b855","creative_name":"Cart Reminder_V2_D3","experiment_uuid":"15a18074-5af2-435d-a373-2c4f22b1ab59","message_uuid":"7cd94a2b-c06a-4c7c-9c79-a9581a413dbe","utm_campaign":"cart-reminder-v2-push","utm_content":"day3","utm_medium":"push-message","utm_source":"blueshift","bucket_uuid":"3e79d631-b572-4c68-a963-8ee8322436de","creative_uuid":"73e454c9-fc6a-4d23-9e24-6e4a1d6d4b02","status_code":1,"status_message":""}']

In [207]:
omnilog['component'][1]

['Jukwaa Web', 100, 102, None, None, None, None, None]

In [206]:
omnilog['event_source'][1]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ckwmul7sg00003r68rxsa0w51',
 'Product - View',
 None,
 None,
 None]